In [25]:
from OCC.Core.BRepTools import breptools_Read
from OCC.Core.TopoDS import *
from OCC.Core.BRep import BRep_Builder
from OCC.Core.gp import *
from OCC.Core.BRepBuilderAPI import *
from math import pi,degrees,radians
from OCC.Core.BRepTools import breptools_Write
import OCCUtils
import OCCUtils.edge
import numpy as np

In [2]:
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer

In [3]:
work_dir = "/home/adam/projects/aluminum_burner/pass_through/hackerCAD/nut_inspection/"

In [4]:
nut = TopoDS_Shape()
builder = BRep_Builder()
assert breptools_Read(nut, '/home/adam/projects/aluminum_burner/pass_through/nut.brep', builder)

In [5]:
trsf = gp_Trsf()
trsf.SetRotation(gp_Ax1(gp_Pnt(0,0,0),gp_Dir(1,0,0)),-pi/2)

In [6]:
mt = BRepBuilderAPI_Transform(nut,trsf)
nut = mt.Shape()

In [89]:
my_renderer = JupyterRenderer()

In [90]:
my_renderer.DisplayShape(nut, render_edges=False)

In [91]:
my_renderer

In [9]:
import OCCUtils

In [92]:
for edge in OCCUtils.Topo(nut).edges():
    a = my_renderer.DisplayShape(edge)

In [93]:
# select the nut in the view
nut_gui = my_renderer.clicked_obj

In [94]:
nut_gui.visible = False

In [ ]:
# get outer edge on top surface
edge_gui = my_renderer.clicked_obj

In [ ]:
edge = my_renderer.GetSelectedShape()

In [ ]:
import OCCUtils.edge

In [ ]:
e = OCCUtils.edge.Edge(edge)

In [ ]:
circ = e.adaptor.Circle()

In [ ]:
r = circ.Radius()

In [ ]:
# select inner edge on top surface in gui
edge2 = my_renderer.GetSelectedShape()

In [ ]:
e2 = OCCUtils.edge.Edge(edge2)

In [ ]:
circ2 = e2.adaptor.Circle()

In [ ]:
r2 = circ2.Radius()

In [ ]:
(r + r2) / 2

In [ ]:
# use r=10 mm for top probing 

In [ ]:
ax1 = circ.Axis()

In [ ]:
ax_new = gp_Ax2(ax1.Location(),ax1.Direction(),gp_Dir(1,0,0))

In [ ]:
circ_new = gp_Circ(ax_new,10)

In [ ]:
me = BRepBuilderAPI_MakeEdge(circ_new,0,2*pi)
edge_for_probe_points = me.Edge()

In [ ]:
my_renderer.DisplayShape(edge_for_probe_points)

In [ ]:
def makeProbePath(pnt):
    p_start = gp_Pnt(pnt.X(),pnt.Y()+1+2,pnt.Z())
    p_end   = gp_Pnt(pnt.X(),pnt.Y()+1-0.5,pnt.Z())
    me = OCC.Core.BRepBuilderAPI.BRepBuilderAPI_MakeEdge(p_start,p_end)
    return me.Edge()

In [ ]:
e = OCCUtils.edge.Edge(edge_for_probe_points)

In [ ]:
curve = e.adaptor.Curve()

In [ ]:
import numpy as np

In [ ]:
edges = []
n_points = 10
delta = 2*pi / n_points
for i in np.linspace(0,2*pi-delta,n_points):
    pnt = curve.Value(i)
    edge = makeProbePath(pnt)
    edges.append(edge)
    my_renderer.DisplayShape(edge)

In [ ]:
help(my_renderer.axes)

In [ ]:
circ.Axis()

In [ ]:
my_renderer

In [34]:
import ModelIKa

In [44]:
client = ModelIKa.IKClient('127.0.0.1',8826)

In [36]:
def solve(v_tip,v_orientation):
    v_tip_meters = v_tip * 1e-3
    v_shank_tip = v_orientation.Normalized() * (0.500+1e-8)
    tup_tip = (v_tip_meters.X(), v_tip_meters.Y(),v_tip_meters.Z())
    tup_shank_tip = v_shank_tip.X(),v_shank_tip.Y(),v_shank_tip.Z()
    x,y,z,a,b = client.solve((*tup_tip,*tup_shank_tip),1e-6,False)
    return x*1e3,y*1e3,z*1e3,a,b

In [ ]:
solve(gp_Vec(1.0,1.0,0),gp_Vec(0,1,0))

In [65]:
gcode = ""
v_tip_to_shank = gp_Vec(0,1,0.0)
for probe_edge in edges:
    e = OCCUtils.edge.Edge(probe_edge)
    curve = e.adaptor.Curve()
    pnt_0 = curve.Value(curve.FirstParameter())
    pnt_1 = curve.Value(curve.LastParameter())
    x_0,y_0,z_0,a_0,b_0 = solve(gp_Vec(pnt_0.XYZ()),v_tip_to_shank)
    gcode += "G01 X{:.6f} Y{:.6f} Z{:.6f} A{:.6f} B{:.6f}\n".format(x_0,y_0,z_0,a_0,b_0)
    x_1,y_1,z_1,a_1,b_1 = solve(gp_Vec(pnt_1.XYZ()),v_tip_to_shank)
    gcode += "G01 X{:.6f} Y{:.6f} Z{:.6f} A{:.6f} B{:.6f}\n".format(x_1,y_1,z_1,a_1,b_1)
    gcode += "G01 X{:.6f} Y{:.6f} Z{:.6f} A{:.6f} B{:.6f}\n".format(x_0,y_0,z_0,a_0,b_0)

In [66]:
print(gcode)

G01 X-6.177514 Y-0.119701 Z27.631005 A90.000005 B1.109386
G01 X-7.619880 Y-0.147534 Z27.631005 A89.999995 B1.109386
G01 X-6.177514 Y-0.119701 Z27.631005 A90.000005 B1.109386
G01 X-5.041171 Y3.522629 Z27.631013 A90.000117 B1.125250
G01 X-6.262511 Y4.365368 Z27.631013 A90.000099 B1.125236
G01 X-5.041171 Y3.522629 Z27.631013 A90.000117 B1.125250
G01 X-2.017898 Y5.809565 Z27.631012 A90.000070 B1.141042
G01 X-2.504904 Y7.219255 Z27.631021 A90.000121 B1.141078
G01 X-2.017898 Y5.809565 Z27.631012 A90.000070 B1.141042
G01 X1.780727 Y5.887140 Z27.630998 A89.999931 B1.156857
G01 X2.215336 Y7.313954 Z27.631021 A90.000122 B1.156895
G01 X1.780727 Y5.887140 Z27.630998 A89.999931 B1.156857
G01 X4.905366 Y3.710141 Z27.630997 A89.999883 B1.172717
G01 X6.085312 Y4.612849 Z27.631014 A90.000106 B1.172705
G01 X4.905366 Y3.710141 Z27.630997 A89.999883 B1.172717
G01 X6.149001 Y0.117926 Z27.631005 A89.999875 B1.188562
G01 X7.620219 Y0.156807 Z27.631005 A90.000013 B1.188538
G01 X6.149001 Y0.117926 Z27.631005 A

In [67]:
preamble = ""
preamble += "G54 G21\n"
preamble += "M6 T1 G43 H1\n"
preamble += "\n"
preamble += "\n"
preamble += "\n"

In [68]:
postamble = "\n\n"
postamble += "M2\n"

In [69]:
f = open("/tmp/bla.ngc","w")
f.write(preamble)
f.write(gcode)
f.write(postamble)
f.close()

In [ ]:
# select edge to create inspection points from
my_renderer.GetSelectedShape()

In [ ]:
nut_gui.material.wireframe = False

In [ ]:
comp = TopoDS_Compound()
builder.MakeCompound(comp)
for e in edges:
    builder.Add(comp,e)

In [ ]:
breptools_Write(comp,"axial_probe_paths.brep")

In [26]:
# create probe paths from circular edge

class ProbePathsFromCircularEdge:
    
    def __init__(self,edge=None):
        
        self.axial_offset_0 = 3
        self.axial_offset_1 = 3
        self.radial_offset_0 = -0.5
        self.radial_offset_1 = -2
        
        self.n_points = 10
        
        if edge:
            self.set_edge(edge)
        
    def set_edge(self,edge):

        self.edge = edge

        self.e = OCCUtils.edge.Edge(edge)

        self.curve = self.e.curve

        self.circ = self.e.adaptor.Circle()
        self.axis = self.circ.Axis()
        self.v_axis = gp_Vec(self.axis.Direction().XYZ())

        self.center_point = self.circ.Position().Location()
        self.v_center = gp_Vec(self.center_point.XYZ())

    def compute(self):
        
        edges = []
        
        delta = 2*pi / self.n_points
        
        for i in np.linspace(0,2*pi-delta,self.n_points):
            pnt_on_edge = self.curve.Value(i)
            v_on_edge = gp_Vec(pnt_on_edge.XYZ())

            v_radial = (v_on_edge - self.v_center).Normalized()

            #start point
            v_start = v_on_edge + self.v_axis*self.axial_offset_0 + v_radial*self.radial_offset_0

            #end point
            v_end = v_on_edge + self.v_axis*self.axial_offset_1 + v_radial*self.radial_offset_1

            me = BRepBuilderAPI_MakeEdge(gp_Pnt(v_start.XYZ()),
                                         gp_Pnt(v_end.XYZ()))
            edge = me.Edge()
            edges.append(edge)
        return edges

In [120]:
edges = []

In [121]:
# select the inner edge of the axial face
pp = ProbePathsFromCircularEdge(my_renderer.GetSelectedShape())
pp.axial_offset_0 = 2
pp.axial_offset_1 = 2
pp.radial_offset_0 = -3
pp.radial_offset_1 = -0.5
id_edges_0 = pp.compute()
edges.extend(id_edges_0)

In [122]:
comp = TopoDS_Compound()
builder.MakeCompound(comp)
for i in id_edges_0:
    builder.Add(comp,i)
breptools_Write(comp,work_dir+"id_edges_0.brep")

True

In [123]:
# select the inner edge of the second axial face
pp = ProbePathsFromCircularEdge(my_renderer.GetSelectedShape())
pp.axial_offset_0 = 2
pp.axial_offset_1 = 2
pp.radial_offset_0 = -3
pp.radial_offset_1 = -0.5
id_edges_1 = pp.compute()
edges.extend(id_edges_1)

In [124]:
comp = TopoDS_Compound()
builder.MakeCompound(comp)
for i in id_edges_1:
    builder.Add(comp,i)
breptools_Write(comp,work_dir+"id_edges_1.brep")

True

In [125]:
# select the inner edge of the axial face
pp = ProbePathsFromCircularEdge(my_renderer.GetSelectedShape())
pp.axial_offset_0 = -3
pp.axial_offset_1 = -0.5
pp.radial_offset_0 = 1.5
pp.radial_offset_1 = 1.5
axial_edges = pp.compute()
edges.extend(axial_edges)

In [126]:
comp = TopoDS_Compound()
builder.MakeCompound(comp)
for i in axial_edges:
    builder.Add(comp,i)
breptools_Write(comp,work_dir+"axial_edges.brep")

True

In [127]:
# select the outer edge of the first axial face
pp = ProbePathsFromCircularEdge(my_renderer.GetSelectedShape())
pp.axial_offset_0 = 2
pp.axial_offset_1 = 2
pp.radial_offset_0 = 3
pp.radial_offset_1 = 0.5
od_edges_0 = pp.compute()
edges.extend(od_edges_0)

In [128]:
comp = TopoDS_Compound()
builder.MakeCompound(comp)
for i in od_edges_0:
    builder.Add(comp,i)
breptools_Write(comp,work_dir+"od_edges_0.brep")

True

In [130]:
for edge in edges:
    my_renderer.DisplayShape(edge)

In [134]:
def probe_edges(edges,v_tip_to_shank):
    gcode = ""
    for probe_edge in edges:
        e = OCCUtils.edge.Edge(probe_edge)
        curve = e.adaptor.Curve()
        pnt_0 = curve.Value(curve.FirstParameter())
        pnt_1 = curve.Value(curve.LastParameter())
        x_0,y_0,z_0,a_0,b_0 = solve(gp_Vec(pnt_0.XYZ()),v_tip_to_shank)
        gcode += "G01 X{:.6f} Y{:.6f} Z{:.6f} A{:.6f} B{:.6f} F100\n".format(x_0,y_0,z_0,a_0,b_0)
        x_1,y_1,z_1,a_1,b_1 = solve(gp_Vec(pnt_1.XYZ()),v_tip_to_shank)
        gcode += "G38.2 X{:.6f} Y{:.6f} Z{:.6f} A{:.6f} B{:.6f} F60\n".format(x_1,y_1,z_1,a_1,b_1)
        gcode += "G01 X{:.6f} Y{:.6f} Z{:.6f} A{:.6f} B{:.6f} F200\n".format(x_0,y_0,z_0,a_0,b_0)
    return gcode

In [135]:
gcode = ""
gcode += "G54 G21\n"
gcode += "M6 T1 G43 H1\n"


gcode += "G90 G0 A90 B0\n"
gcode += "G53 G90 G0 Z0\n"
gcode += "G0 X0 Y0\n"
gcode += "G0 Z27.63\n"

v_tip_to_shank = gp_Vec(0,1,0)
gcode += probe_edges(id_edges_0,v_tip_to_shank)

gcode += "G01 X0 Y0\n"
gcode += "G01 Z22.617\n"

gcode += probe_edges(id_edges_1,v_tip_to_shank)

gcode += "G01 X0 Y0\n"
gcode += "G01 Z31.65\n"

gcode += probe_edges(axial_edges,v_tip_to_shank)

gcode += "G01 X-14.01 Y-0.396\n"

gcode += probe_edges(od_edges_0,v_tip_to_shank)

gcode += "G53 G90 G0 Z0\n"
gcode += "M2\n"


In [136]:
f = open("/tmp/bla1.ngc","w")
f.write(gcode)
f.close()